
## Search through StructureDefinition properties

*USE Python 3.7+ to maintain order of Json files*

- Fetch URLs from excel file
- Fetch published URl
- search for SD elements

### import python modules

In [2]:
from pathlib import Path
import pandas as pd
import csv
from json import loads, dumps
from requests import get,post
from fhir.resources.structuredefinition import StructureDefinition
from pprint import pprint

headers = {'Accept': 'application/fhir+json', 'Content-Type': 'application/fhir+json'}
in_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/my-notes/MedReq-MedDisp-profiles.csv')
in_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/my-notes/SD_URL.md.csv')
fsh_project = '/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project'
sushi_config_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project/sushi-config.yml')
fsh_file_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project/input/fsh/file.fsh')
fsh_gen_resource_path = Path('/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project/fsh-generated/resources')

in_path

PosixPath('/Users/ehaas/Documents/FHIR/US-Core/my-notes/SD_URL.md.csv')

### Loop through SDs
 - read sheet as DF
 - iterate through urls
   - fetch SD as xml, json, or fsh
   - convert all to python fhir object
   - list id

In [3]:
test_type = 'MedicationRequest' #'ServiceRequest'
test_me =["MedicationRequest.reasonCode","MedicationRequest.reasonReference",]#["ServiceRequest.reasonCode","ServiceRequest.reasonReference",]
#test_me =["Coverage.period",]
count1 = 1
count2 = 1
list1 = []
list2 = []

def check_element(sd):
    global count1, count2, list1, list2
    if sd.type == test_type:
        print(f'({count1}) Type = {test_type}')
        count1 += 1
        list1.append(sd.url)
        print(f'sd.id = {sd.id}')
        element_paths = [e.path for e in sd.differential.element]
        if any(item in element_paths for item in test_me):
                print(f'>>>>>>>>>({count2}) reason[x] element found!!<<<<<<<<<<')
                list2.append(sd.url)
                count2 += 1

                
def update_sushi(my_file):
    '''update the sushi-config.yaml file'''
    project_code  = my_file.split('/')[4:5][0]
    print(f'project_code: {project_code}')
    sushi_config = f'''
FSHOnly: true
fhirVersion: 4.0.1
canonical: "http://hl7.org/fhir/us/{project_code}"
name: Temp
status: draft
''' 
    # print(sushi_config)
    # print(config_path)
    sushi_config_path.write_text(sushi_config)

def make_fhir_object():
    for file in fsh_gen_resource_path.glob('*.json'):
        print(file)
        try:
            sd = StructureDefinition.parse_file(file)
            file.unlink()
        except:
            print(f'Error parsing {my_file}:\n {response.text[:75]}')
            file.unlink()
        else:
            return sd


df = pd.read_csv(in_path, header=0, encoding='utf-8')
for i,my_file in enumerate(df.url):
    print(f'*************************************\n{i+1}: {my_file}\n*************************************')
    with get(my_file, headers=headers) as response:
        if response.status_code == 200:
            if my_file.split('.')[-1] == 'xml':
                data = response.text.replace('<?xml version="1.0" encoding="UTF-8"?>','',1) # check all spellings!!!
                try:       
                    sd = StructureDefinition.parse_raw(data, content_type="text/xml")
                except:
                    print(f'Error parsing {my_file}:\n {response.text[:75]}')
                else:
                    check_element(sd)
            if my_file.split('.')[-1]  == 'json':
                try:
                    sd = StructureDefinition.parse_obj(response.json())
                except:
                    print(f'Error parsing {my_file}:\n {response.text[:75]}')
                else:
                    check_element(sd)
            if my_file.split('.')[-1]  == 'fsh':
                print('convert fsh to json')
                update_sushi(my_file)
                fsh_file_path.write_text(response.text)
                #build json from fsh files using sushi
                !sushi $fsh_project > "/Users/ehaas/Documents/Python/Jupyter/sushi.txt"
                sd = make_fhir_object()
                try:
                    check_element(sd)
                except AttributeError:
                    print(f'Error parsing {my_file}:\n sd = {sd}')

print(f'Total {test_type} Profiles = {count1-1}')
pprint(list1)
print(f'Total {test_type} Profiles with {test_me} in differential = {count2-1}')
pprint(list2)
print(f'Total Files = {i+1}')
                




*************************************
1: https://raw.githubusercontent.com/HL7/dental-data-exchange/6bbcfe5f82492b1de0569d59490766b21c4b5119/input/resources/structuredefinition/dental-servicerequest.xml
*************************************
(1) Type = ServiceRequest
sd.id = dental-servicerequest
>>>>>>>>>(1) reason[x] element found!!<<<<<<<<<<
*************************************
2: https://raw.githubusercontent.com/HL7/case-reporting/23bdb08d27747fde62476b1f18b1f76bdfb97a94/input/resources/structuredefinition/eicr-servicerequest.xml
*************************************
(2) Type = ServiceRequest
sd.id = eicr-servicerequest
*************************************
3: https://raw.githubusercontent.com/HL7/sdc/f9c12c2eb322f3e97d4c9baf54d78fd17269baaa/input/resources/StructureDefinition-sdc-servicerequest.xml
*************************************
(3) Type = ServiceRequest
sd.id = sdc-servicerequest
>>>>>>>>>(2) reason[x] element found!!<<<<<<<<<<
*************************************
4: htt

In [6]:


    with get(i, headers=headers) as response:
        if response.status_code == 200:
     
            try:
                if response.json()['type'] == 'ServiceRequest':
                    r_json = response.json()
                    
            except JSONDecodeError:
                print(f'>>>>>>>>>>>>>>>>JSONDecodeError: {i}')
                print(response.text[:75])
                data = response.text.replace('<?xml version="1.0" encoding="UTF-8"?>','',1) # check all spellings
                print(data[:75])
                try:
                    r_json = StructureDefinition.parse_raw(data, content_type="text/xml")
                except:
                    print('XML Parse Failed')
                    print(response.text[:75])
                    break
                print(f'{r_json.id}\n <<<<<<<<<<<<<<<<<<<')
        
            except KeyError:
                    print(f'KeyError: {i}')
                    print(response.json()['resourceType'])
            else:
                print(f'{count}: {i}')
                count += 1
                element_paths = [e["path"] for e in r_json["differential"]["element"]]
                if any(item in element_paths for item in test_me):
                        print(f'indication element found!!')
                        # my_element = [e for e in response.json()["differential"]["element"] if e["path"] in test_me]
                        # print(f'my_element = {my_element}')
        else:
            print(f'{count}: {i} is not valid')
            count += 1
        

SyntaxError: 'break' outside loop (408563504.py, line -1)

loop through all sd and print out the element the element you want

filter on the element properties you want

In [ ]:
def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}', encoding="utf8") as f:
        r = f.read()
        return(json.loads(r))
constraints = {}
sps = []
test_resources = [
"CareTeam",
"Condition",
"DiagnosticReport",
"Encounter",
"Goal",
"Location",
"Observation",
"Organization",
"Patient",
"Practitioner",
"PractitionerRole",
"Procedure",
"RelatedPerson",
"ServiceRequest",
]
for i in us_core_files: 
    
    sp_dict = open_file(us_core,i)
    
    try:
        assert sp_dict['base'][0] in test_resources
        #sps.append({"base":sp_dict['base'][0],"code":sp_dict['code']})
        sps.append(sp_dict['base'][0]  + " `" + sp_dict['code'] + "` search parameter ")
    except:
        pass

for sp in sorted(sps):
    print("   - "+sp)